# SceneXplain: Alt-text generator

In this notebook we will use Jina AI's [SceneXplain](https://scenex.jina.ai) to cycle through a zip file of images and generate an alt-text for each one. The output will be stored in a CSV file then re-zipped with the images.

For example, if we send the image file: `free-photo-of-red.jpeg`:

![](https://github.com/alexcg1/notebooks/blob/main/scenex/a11y-alt-tags/example/free-photo-of-red.jpeg?raw=true)

We get the output:

```json
{
  "alt_text": "Rear view of a red classic 'Malibu Classic' car focusing on tail light and chrome bumper"
}
```

We will:

- Unzip the images
- Send them to SceneXplain to get alt-texts
- Write the alt-texts to a CSV file, along with the image filenames
- Re-zip the file

Before proceeding, please sign up for [SceneXplain](https://scenex.jina.ai), create an [API key](https://scenex.jina.ai/api), and paste it below:

In [25]:
SCENEX_SECRET = "52vxFUyvkMw8Gj0mANeI:945e07e84e19b98953e85654a49d096ec49416da321e9b91f50171cbc75b03b0"

## Notes

### File structure

The zip file must have a flat structure. That is, all image files must be at the top level of the file, e.g:

```
images.zip
├── free-photo-of-a-bowl-of-granola-with-fruit-and-nuts-on-a-wooden-cutting-board.jpeg
├── free-photo-of-alexandrine-parakeet-in-side-view.png
├── free-photo-of-holida-christmas-party-drinks-ornaments.jpeg
├── free-photo-of-leaves-on-the-branch.jpeg
├── free-photo-of-pose-woman-dress-in-the-desert-gold-light-curly-hair.jpeg
├── free-photo-of-red.jpeg
└── pexels-photo-12015253.png
```

### Free plan limitations

Many users of this notebook may be using the free SceneXplain plan, which is limited to eight images per batch. For the sake of this example, we suggest you use [this zipfile of seven images](https://github.com/alexcg1/notebooks/raw/main/scenex/a11y-alt-tags/images.zip). For more complex needs, you may need to either refactor the code in this notebook or switch to a [paid plain](https://scenex.jina.ai/#pricing).

## SceneXplain JSON Store

We'll use SceneXplain's JSON Store to pull a predefined schema that is specialized for generating alt-texts. This is just a simple use case for the JSON Store, so be sure to browse and explore what else you can do with it!

Our JSON Schema looks something like this:

```json
{
  "type": "object",
  "properties": {
    "alt_text": {
      "type": "string",
      "description": "the most concise description possible of the image’s purpose. If the image is purely decorative (e.g. part of the website's design, not content), leave empty. Do not include text like 'this image contains' or 'image depicts'"
    }
  }
}
```

Below we've defined two JSON Schema IDs, one for general alt-texts, and one specifically targeted for screenshot alt-texts.

In [26]:
json_schema = "z2qaOJPkkR7k8wr2OV1q"  # generic alt-texts
# json_schema = "WJ84ezfnag0zWxgkmDJY"  # screenshot alt-texts

## User options

Some basic options about input/output/CSV filenames.

In [27]:
# filenames
zip_input = "images.zip"
zip_output = "images_with_alt_texts.zip"
csv_filename = "alt_texts.csv"

## Basic setup

Functions to send generate image lists, populate the data to send to SceneXplain, send that data.

In [28]:
import os
import json
import base64
import http.client
from pprint import pprint
from glob import glob
import csv
import tempfile

In [29]:
def extract_zip(zip_file):
    temp_dir = tempfile.mkdtemp()
    with zipfile.ZipFile(zip_input, 'r') as zip_ref:
        zip_ref.extractall(temp_dir)

        return temp_dir

In [30]:
def generate_image_list(folder_name, max_count=100):
    filetypes = ['jpg', 'jpeg', 'png']
    image_files = []
    
    for filetype in filetypes:
        image_files.extend(glob(f'{folder_name}/*.{filetype}'))

    return image_files[:max_count]

In [31]:
# SceneX setup and functions

scenex_headers = {
    "x-api-key": f"token {SCENEX_SECRET}",
    "content-type": "application/json",
}

def image_to_data_uri(file_path):
    with open(file_path, "rb") as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode("utf-8")
        return f"data:image/png;base64,{encoded_image}"
        
def generate_scenex_data(image_files):
    data = {}
    data['data'] = []

    for file in image_files:
        cid = file.split('/')[-1]
        row = {
            "image": image_to_data_uri(file),
            "features": ["json"],
            "json_schema_id": json_schema
        }

        data['data'].append(row)

    return data

def process_scenex(data):
    connection = http.client.HTTPSConnection("api.scenex.jina.ai")
    connection.request("POST", "/v1/describe", json.dumps(data), scenex_headers)
    response = connection.getresponse()
    response_data = response.read().decode("utf-8")
    
    connection.close()

    return json.loads(response_data)['result']

## Creating our alt-texts

### Unzip images

Unzip the images into a temporary directory which we'll clean up later. For this notebook we'll use an example zipfile of images.

In [32]:
if not os.path.isfile(zip_input):
    print("Downloading images.zip")
    os.system("wget -q https://github.com/alexcg1/notebooks/raw/main/scenex/a11y-alt-tags/images.zip")

In [33]:
import zipfile

In [34]:
temp_dir = extract_zip(zip_input)

### Generate data

Let's take the image list and add our SceneXplain options (e.g. JSON Schema ID, image data URI, etc).

In [35]:
image_files = generate_image_list(temp_dir)

In [36]:
scenex_data = generate_scenex_data(image_files)

### Send data to SceneXplain

In [37]:
scenex_response = process_scenex(scenex_data)

### Write to CSV

In [38]:
headers = ["filename", "alt-text"]

In [39]:
def generate_csv(csv_filename, image_files, scenex_response):
    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        for filename, row in zip(image_files, scenex_response):
            alt_text = json.loads(row['i18n']['en'])['alt_text']
            row = [filename, alt_text]
            writer.writerow(row)

In [40]:
generate_csv(csv_filename, image_files, scenex_response)

### View results

Let's see how SceneXplain's alt-texts look:

In [41]:
!cat alt_texts.csv

filename,alt-text
/tmp/tmpwg57y6yw/free-photo-of-red.jpeg,A close-up of a red Malibu Classic car parked with focus on the rear badge.
/tmp/tmpwg57y6yw/free-photo-of-pose-woman-dress-in-the-desert-gold-light-curly-hair.jpeg,Woman with curly hair in a brown coat and black top standing on a beach during sunset.
/tmp/tmpwg57y6yw/free-photo-of-leaves-on-the-branch.jpeg,"Close-up of a branch with green and yellow leaves against a backdrop of green plants, indicating the arrival of autumn."
/tmp/tmpwg57y6yw/free-photo-of-holida-christmas-party-drinks-ornaments.jpeg,"Festive Christmas setup with red tablecloth, green velvet curtain, two empty crystal glasses, and decorations including candy canes and golden ornaments."
/tmp/tmpwg57y6yw/free-photo-of-a-bowl-of-granola-with-fruit-and-nuts-on-a-wooden-cutting-board.jpeg,Bowl of granola with pomegranate seeds and sliced strawberries on a wooden cutting board against a dark background
/tmp/tmpwg57y6yw/pexels-photo-12015253.png,"An old-fashioned, re

### Re-zip the files

In [42]:
with zipfile.ZipFile(zip_output, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(csv_filename)

    for foldername, subfolders, filenames in os.walk(temp_dir):
        for filename in filenames:
            file_path = os.path.join(foldername, filename)
            zipf.write(file_path, os.path.relpath(file_path, temp_dir))

If you're using Colab you can now right-click the zip file in your sidebar to download it.

### Clean up

In [43]:
import shutil
os.remove(csv_filename)
shutil.rmtree(temp_dir)